libraries

In [138]:
!pip install roboflow
!pip install transformers
!pip install torchvision

zip upload to g colab or manually pth upload

In [139]:
# from google.colab import files
# uploaded_file = files.upload()
# cancel when rerunning because another upload isn't needed when tuning // uncomment when needed

trying api instead of preprocessing zip, not cleaned

In [140]:
from roboflow import Roboflow
rf = Roboflow(api_key="ZSy2TaHq3naWbRd02t56")
project = rf.workspace("model3").project("model3-2tttc")
version = project.version(2)
dataset = version.download("folder")

loading Roboflow workspace...
loading Roboflow project...


imported libraries

In [141]:
from roboflow import Roboflow
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import ViTForImageClassification, ViTFeatureExtractor
import torch
from torch import nn, optim
from google.colab import files
import zipfile
import os
import random
import numpy as np
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)


unzip

In [142]:
# robo flow "Food Ingredient Recognition Computer Vision Project" dataset
"""
zip_path = "Food Ingredient Recognition.v4i.multiclass.zip"
extract_path = "model3_dataset"

if not os.path.exists(extract_path):
    print("Extracting-----")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print("Extraction done")
else:
    print("data already extracted")
"""


'\nzip_path = "Food Ingredient Recognition.v4i.multiclass.zip"\nextract_path = "model3_dataset"\n\nif not os.path.exists(extract_path):\n    print("Extracting-----")\n    with zipfile.ZipFile(zip_path, \'r\') as zip_ref:\n        zip_ref.extractall(extract_path)\n    print("Extraction done")\nelse:\n    print("data already extracted")\n'

data

In [143]:
transform = transforms.Compose([transforms.Resize((224, 224)),transforms.ToTensor()])

try:
    data_dir = dataset.location
except NameError:
    data_dir = "model3_dataset"

train_dataset = datasets.ImageFolder(f"{data_dir}/train", transform=transform)
valid_dataset = datasets.ImageFolder(f"{data_dir}/valid", transform=transform)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16)


pth

In [144]:
NUM_CLASSES = len(train_dataset.classes)

model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224',
    num_labels=NUM_CLASSES,
    ignore_mismatched_sizes=True
)

if os.path.exists("vit_model3.pth"):
    model.load_state_dict(torch.load("vit_model3.pth"))
    print("trained model loaded")
else:
    print("vit_model3.pth not found")
    from google.colab import files
    uploaded = files.upload()
    model.load_state_dict(torch.load("vit_model3.pth"))

model = model.to("cuda" if torch.cuda.is_available() else "cpu")


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([577]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([577, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trained model loaded


was drive, unneeded

In [145]:
# from google.colab import drive
# drive.mount('/content/drive')

switch to CPU if GPU runs out of usage

In [146]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vision transformer model , ViT

In [147]:
NUM_CLASSES = len(train_dataset.classes)
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224', num_labels=NUM_CLASSES, ignore_mismatched_sizes=True)
model.load_state_dict(torch.load("vit_model3.pth"))
# make sure runs gpu
model = model.to("cuda" if torch.cuda.is_available() else "cpu")

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([577]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([577, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [148]:
from collections import Counter
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# training label
all_labels = []
for _, labels in train_loader:
    all_labels.extend(labels.tolist())

# classweigh t
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(all_labels),
    y=np.array(all_labels)
)

class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

optimizer = optim.Adam(model.parameters(), lr=3e-4)
# make sure runs gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

manual seed from best trial

In [149]:
# fixed random seed for reproducibility
import torch, numpy as np, random
seed = 42617
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)
print(f"Best (MANUAL) Seed: {seed}")


Best (MANUAL) Seed: 42617


training

In [150]:
# finding random seed until good one
"""
# random seed
import torch, numpy as np, random
seed = random.randint(0, 99999)
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

print(f"Reinitialized seed: {seed}")

# reinitialize model
NUM_CLASSES = len(train_dataset.classes)
model = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224',
    num_labels=NUM_CLASSES,
    ignore_mismatched_sizes=True
)
"""
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

# # of epochs
for epoch in range(6):
    model.train()
    total_loss = 0

    for batch_idx, (imgs, labels) in enumerate(train_loader):
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs).logits
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

        if (batch_idx + 1) % 10 == 0:
            print(f"Epoch #{epoch+1}, Batch {batch_idx+1} Loss: {loss.item():.4f}")

    print(f"Epoch #{epoch+1} finished // Total loss: {total_loss:.4f}")

# trial save if good
torch.save(model.state_dict(), "vit_model3_trial.pth")
print("Trial saved as vit_model3_trial.pth")


Epoch #1, Batch 10 Loss: 6.2630
Epoch #1, Batch 20 Loss: 5.9186
Epoch #1, Batch 30 Loss: 5.6811
Epoch #1, Batch 40 Loss: 5.8674
Epoch #1, Batch 50 Loss: 6.8033
Epoch #1, Batch 60 Loss: 5.6405
Epoch #1, Batch 70 Loss: 6.2712
Epoch #1, Batch 80 Loss: 7.0290
Epoch #1, Batch 90 Loss: 6.5769
Epoch #1, Batch 100 Loss: 7.1161
Epoch #1, Batch 110 Loss: 6.3455
Epoch #1, Batch 120 Loss: 6.0799
Epoch #1, Batch 130 Loss: 5.7025
Epoch #1, Batch 140 Loss: 7.0919
Epoch #1, Batch 150 Loss: 6.1246
Epoch #1, Batch 160 Loss: 5.9975
Epoch #1, Batch 170 Loss: 6.3435
Epoch #1, Batch 180 Loss: 5.9538
Epoch #1, Batch 190 Loss: 5.9026
Epoch #1, Batch 200 Loss: 6.8525
Epoch #1, Batch 210 Loss: 6.3617
Epoch #1, Batch 220 Loss: 6.2015
Epoch #1, Batch 230 Loss: 6.3747
Epoch #1, Batch 240 Loss: 6.6255
Epoch #1, Batch 250 Loss: 6.1682
Epoch #1, Batch 260 Loss: 5.7625
Epoch #1, Batch 270 Loss: 6.1624
Epoch #1, Batch 280 Loss: 5.8354
Epoch #1, Batch 290 Loss: 5.5894
Epoch #1, Batch 300 Loss: 5.9089
Epoch #1, Batch 310

model save for safety in case g colab runs out

In [151]:
torch.save(model.state_dict(), "vit_model3.pth")

model load and save

In [152]:
model.load_state_dict(torch.load("vit_model3.pth"))
model = model.to(device)

model eval

In [153]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for imgs, labels in valid_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        outputs = model(imgs).logits
        _, preds = torch.max(outputs, 1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

accuracy = correct / total
print(f"Accuracy: {accuracy:.6%}")

Accuracy: 0.000000%


safety dwnload 1

In [154]:
from google.colab import files
files.download("vit_model3_epoch3.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

safety dwnload 2

In [155]:

from google.colab import files
files.download("vit_model3.pth")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

actual vs pred lables

In [156]:
imgs, labels = next(iter(valid_loader))
imgs, labels = imgs.to(device), labels.to(device)
outputs = model(imgs).logits
_, preds = torch.max(outputs, 1)

print("Predictions:", preds[:10])
print("Actual labels:", labels[:10])


Predictions: tensor([320, 320, 382, 320, 320, 320, 320, 320, 320, 320], device='cuda:0')
Actual labels: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 1], device='cuda:0')


prediction checker

In [157]:
from collections import Counter
all_preds = []

model.eval()
with torch.no_grad():
    for imgs, _ in valid_loader:
        imgs = imgs.to(device)
        outputs = model(imgs).logits
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())

print(Counter(all_preds))


Counter({np.int64(320): 369, np.int64(158): 111, np.int64(533): 96, np.int64(212): 92, np.int64(471): 17, np.int64(452): 9, np.int64(280): 7, np.int64(382): 2, np.int64(29): 2, np.int64(541): 1, np.int64(144): 1})
